In [8]:
import pandas as pd
import numpy as np
import datetime
import gc
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm
import xgboost as xgb
from sklearn import linear_model

In [3]:
TRAIN_MONTH = '2015_05_28'
TEST_MONTH = '2016_05_28'

TRAIN_FILE = 'data/train_' + TRAIN_MONTH + '.csv'
ADDED_PRODUCTS_FILE = 'data/added_product_' + TRAIN_MONTH + '.csv'

TEST_FILE = 'data/train_' + TEST_MONTH + '.csv'

HEADER = ["fecha_dato", "ncodpers", "ind_empleado",
          "pais_residencia", "sexo", "age", "fecha_alta",
          "ind_nuevo", "antiguedad", "indrel", "ult_fec_cli_1t",
          "indrel_1mes", "tiprel_1mes", "indresi", "indext",
          "conyuemp", "canal_entrada", "indfall", "tipodom",
          "cod_prov", "nomprov", "ind_actividad_cliente",
          "renta", "segmento", "ind_ahor_fin_ult1",
          "ind_aval_fin_ult1", "ind_cco_fin_ult1",
          "ind_cder_fin_ult1", "ind_cno_fin_ult1",
          "ind_ctju_fin_ult1", "ind_ctma_fin_ult1",
          "ind_ctop_fin_ult1", "ind_ctpp_fin_ult1",
          "ind_deco_fin_ult1", "ind_deme_fin_ult1",
          "ind_dela_fin_ult1", "ind_ecue_fin_ult1",
          "ind_fond_fin_ult1", "ind_hip_fin_ult1",
          "ind_plan_fin_ult1", "ind_pres_fin_ult1",
          "ind_reca_fin_ult1", "ind_tjcr_fin_ult1",
          "ind_valo_fin_ult1", "ind_viv_fin_ult1",
          "ind_nomina_ult1", "ind_nom_pens_ult1",
          "ind_recibo_ult1"]

In [4]:
train = pd.read_csv(TRAIN_FILE, header=None, names=HEADER)
test = pd.read_csv(TEST_FILE, header=None, names=HEADER)
added_products = pd.read_csv(ADDED_PRODUCTS_FILE)

combined = pd.concat((train, test)).reset_index(drop=True)

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
def process_data_from_original_dataframe(combined):
    # fixing age
    combined['age'] = pd.to_numeric(combined['age'], errors='coerce')

    combined.loc[combined.age < 18, "age"] = \
        combined.loc[(combined.age > 18) &
                     (combined.age <= 30), "age"].mean(skipna=True)
    combined.loc[combined.age > 100, "age"] = \
        combined.loc[(combined.age > 30) &
                     (combined.age <= 100), "age"].mean(skipna=True)
    combined['age'].fillna(combined['age'].mean(), inplace=True)
    combined['age'] = combined['age'].astype(int)

    # fix ind_nuevo..
    combined.loc[combined.ind_nuevo.isnull(), 'ind_nuevo'] = 1

    # fix antiguedad
    combined['antiguedad'] = pd.to_numeric(combined['antiguedad'],
                                           errors='coerce')
    combined.loc[combined.antiguedad.isnull(), 'antiguedad'] = \
        combined.antiguedad.min()
    combined.loc[combined.antiguedad < 0, 'antiguedad'] = 0

    # fix indrel
    combined.loc[combined.indrel.isnull(), 'indrel'] = 1

    # drop useless cols
    combined.drop(['tipodom', 'cod_prov'], axis=1, inplace=True)

    # fix ind_actividad_cliente
    combined.loc[combined.ind_actividad_cliente.isnull(),
                 "ind_actividad_cliente"] = \
        combined.ind_actividad_cliente.median()

    # fix city name
    combined.loc[combined.nomprov ==
                 "CORU\xc3\x91A, A", "nomprov"] = "CORUNA, A"
    combined.loc[combined.nomprov.isnull(), 'nomprov'] = 'UNKNOWN'

    # fix incomes
    # combined.renta = pd.to_numeric(combined.renta, errors='coerce')
    grouped = combined.groupby('nomprov').\
        agg({'renta': lambda x: x.median(skipna=True)}).reset_index()
    new_incomes = pd.merge(combined, grouped,
                           how='inner',
                           on='nomprov').loc[:, ['nomprov', 'renta_y']]

    new_incomes = new_incomes.\
        rename(columns={"renta_y": "renta"}).\
        sort_values("renta").sort_values("nomprov")

    combined.sort_values("nomprov", inplace=True)
    combined = combined.reset_index()
    new_incomes = new_incomes.reset_index()
    combined.loc[combined.renta.isnull(), "renta"] = \
        new_incomes.loc[combined.renta.isnull(), "renta"].median()
    combined.sort_values(by='fecha_dato', inplace=True)

    # rest of the columns
    string_data = combined.select_dtypes(include=["object"])
    missing_columns = [col for col in string_data
                       if string_data[col].isnull().any()]
    del string_data

    combined.loc[combined.indfall.isnull(), 'indfall'] = 'N'
    combined.loc[combined.tiprel_1mes.isnull(), 'tiprel_1mes'] = 'A'
    combined.tiprel_1mes = combined.tiprel_1mes.astype('category')

    map_dict = {
        '1.0': '1',
        '1': '1',
        '3.0': '3',
        'P': 'P',
        3.0: '3',
        2.0: '2',
        '3': '3',
        '2.0': '2',
        '4.0': '4',
        '4': '4',
        '2': '2',
        1.0: '1',
        4.0: '4'
    }

    combined.indrel_1mes.fillna('P', inplace=True)
    combined.indrel_1mes = combined.indrel_1mes.apply(lambda x: map_dict[x])
    combined.indrel_1mes = combined.indrel_1mes.astype('category')

    unknown_cols = [col for col in missing_columns if
                    col not in ['indfall', 'tiprel_1mes', 'indrel_1mes']]
    for col in unknown_cols:
        combined.loc[combined[col].isnull(), col] = "UNKNOWN"

    # feature cols
    feature_cols = combined.iloc[:1, ].filter(regex="ind_+.*ult.*").\
        columns.values
    for col in feature_cols:
        combined.loc[combined[col].isnull(), col] = 0
        combined[col] = combined[col].astype(int)

    del combined['ult_fec_cli_1t'], combined['fecha_alta']

    encoders = []
    for col in ['sexo', 'indrel_1mes', 'pais_residencia', 'ind_empleado',
                'segmento', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp',
                'canal_entrada', 'indfall', 'nomprov']:
        temp_enc = LabelEncoder()
        temp_enc.fit(combined[col])
        combined[col] = temp_enc.transform(combined[col])
        encoders.append(temp_enc)
    return combined, encoders

In [6]:
combined, encoders = process_data_from_original_dataframe(combined)

In [18]:
train = combined.loc[combined.fecha_dato == '2015-05-28', :].reset_index(drop=True)
test = combined.loc[combined.fecha_dato == '2016-05-28', :].reset_index(drop=True)

del train['index'], test['index']
del train['fecha_dato'], test['fecha_dato']

#added_products.set_index('ncodpers', inplace=True)

label_encoder = LabelEncoder()
label_encoder.fit(added_products['added_product'])
added_products['encoded_products'] = label_encoder.transform(added_products['added_product'])

In [23]:
logistic = linear_model.LogisticRegression(verbose=1)
# train.set_index('ncodpers', inplace=True)
xTrain = train.loc[added_products.index, :]
yTrain = added_products.loc[:, 'encoded_products']
logistic.fit(train.loc[added_products.index, :], added_products.loc[:, 'encoded_products'])

[LibLinear]

/usr/local/lib/python2.7/site-packages/sklearn/svm/base.py:924: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=1, warm_start=False)

In [28]:
logistic.score(xTrain, yTrain)

0.24460414420888729

In [25]:
logistic.predict_proba(xTrain.head())

array([[  7.88388596e-02,   1.22389780e-05,   1.23226542e-02,
          5.71197460e-07,   1.46024128e-08,   2.10457137e-02,
          7.64390653e-03,   1.33830336e-21,   2.24665410e-02,
          1.06971050e-03,   9.09661108e-03,   9.10647868e-03,
          1.37389017e-04,   3.02297621e-01,   9.26104628e-02,
          2.37162201e-04,   3.43347582e-04,   6.89041060e-02,
          1.68434847e-01,   1.99760659e-01,   5.67013746e-03,
          9.68000644e-07],
       [  8.28637576e-02,   1.38160245e-04,   1.28689125e-02,
          6.13483200e-07,   1.39945123e-08,   2.44613962e-02,
          6.93007965e-03,   1.64940317e-21,   2.24217309e-02,
          1.55160060e-03,   8.39048407e-03,   8.70945941e-03,
          1.37553518e-04,   2.88636274e-01,   9.21249089e-02,
          2.35374359e-04,   3.13801654e-04,   7.08112724e-02,
          1.62059299e-01,   2.11927563e-01,   5.36560602e-03,
          5.21388997e-05],
       [  9.67688740e-02,   4.18382811e-04,   1.66309329e-02,
          2.9259

In [26]:
test.set_index('ncodpers', inplace=True)
preds = logistic.predict_proba(test)

top_t_products = label_encoder.inverse_transform(np.argsort(preds, axis = 1)[:, ::-1][:, :])

test['xgb_preds'] = [' '.join(x) for x in top_t_products]
test['added_products'] = ['ind_recibo_ult1']*test.shape[0]

for i in tqdm(test.index):
    products = map(lambda x: x[0], filter(lambda x: x[1]==1, zip(HEADER[24:], test.loc[i, 'ind_ahor_fin_ult1':'ind_recibo_ult1'])))
    pred_products = test.loc[i, 'xgb_preds'].split()
    prod_string = ' '.join(filter(lambda x: x not in products, pred_products))
    test.set_value(i, 'added_products', prod_string)

submission = pd.read_csv('data/test_ver2.csv', usecols=[1])
submission['added_products'] = ['ind_recibo_ult1']*submission.shape[0]
submission.set_index('ncodpers', inplace=True)
submission.added_products = test.loc[submission.index, 'added_products']

100%|██████████| 931453/931453 [18:19<00:00, 847.49it/s]


In [27]:
from scripts.kaggle.helpers import make_submission
filename = 'data/submissions/4.my_first_logistic_sub_trained_on_jun_2015_only_added_users.csv'
description = 'my first logistic submission on jun 2015 only added products data, default logistic params, and original xgb features'
submission.to_csv(filename, columns=['added_products'])
make_submission(filename, description=description, submit=True, compress=True)
# make_submission('data/submissions/2_bayesian_prob_products_counters_with_old_1_ignored.csv', description="2. based on bayesian probabilities, where probs are calculated only for prods when there is a transition from 0 to 1", submit=True, compress=True)


Compressing file
Compression done
Uploading submission data/submissions/4.my_first_logistic_sub_trained_on_jun_2015_only_added_users.csv.zip
Upload done
